In [ ]:
#new ipynb

In [1]:
!git clone https://github.com/neheller/kits23
%cd kits23
!pip3 install -e .

Cloning into 'kits23'...
remote: Enumerating objects: 15613, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 15613 (delta 42), reused 62 (delta 30), pack-reused 15526 (from 1)
Receiving objects: 100% (15613/15613), 493.94 MiB | 43.06 MiB/s, done.
Resolving deltas: 100% (14011/14011), done.
Updating files: 100% (5720/5720), done.
/content/kits23
Obtaining file:///content/kits23
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/deepmind/surface-distance.git to /tmp/pip-install-7avuh4dc/surface-distance_c87575428c014946a2b7eabf15983b12
  Running command git clone --filter=blob:none --quiet https://github.com/deepmind/surface-distance.git /tmp/pip-install-7avuh4dc/surface-distance_c87575428c014946a2b7eabf15983b12
  Resolved https://github.com/deepmind/surface-distance.git to commit 1f805cea446805e559badc80dd27f34175e4173d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!kits23_download_data


Found 489 cases to download

Dowloading case_00588...: 100% 489/489 [09:11<00:00,  1.13s/it]


In [2]:
# 1. Restart runtime after installing if prompted by Colab
!pip install -q "torch" torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q monai
!pip install -q nibabel scikit-image pandas tqdm SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.9 MB/s eta 0:00:00


In [5]:
import os, json, glob
import random

root = "/content/kits23/dataset"

# Get all case folders
case_folders = sorted([d for d in glob.glob(os.path.join(root, "case_*")) if os.path.isdir(d)])

datalist = {"training": []}

num_folds = 5
random.seed(42)  # for reproducibility

for case_path in case_folders:
    case_id = os.path.basename(case_path)

    # Paths
    label_path = os.path.join(case_path, "segmentation.nii.gz")
    image_path = os.path.join(case_path, "imaging.nii.gz")

    if not os.path.exists(label_path) or not os.path.exists(image_path):
        print(f"Skipping {case_id}, missing image or label")
        continue

    # Random fold assignment
    fold = random.randint(0, num_folds - 1)

    datalist["training"].append({
        "image": os.path.relpath(image_path, start=root),
        "label": os.path.relpath(label_path, start=root),
        "image_filename": os.path.basename(image_path),
        "fold": fold
    })

dataset_json = {
    "name": "KiTS23_custom_folds",
    "description": "KiTS23 formatted dataset with random 5-fold split",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "modality": {"0": "CT"},
    "labels": {"0": "background", "1": "kidney_and_mass", "2":"mass", "3":"tumor"},
    "numTraining": len(datalist["training"]),
    "training": datalist["training"],
    "test": []
}

with open(os.path.join(root, "dataset.json"), "w") as f:
    json.dump(dataset_json, f, indent=2)

print("Wrote dataset.json with", len(datalist["training"]), "cases and random fold assignments")


Wrote dataset.json with 489 cases and random fold assignments


In [6]:
pip install fire

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 4.9 MB/s eta 0:00:00


In [10]:
!python -m monai.apps.auto3dseg AutoRunner run \
    --input "/content/kits23/input.yaml" \
    --algos segresnet


2025-10-16 13:40:51.766636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760622052.041587   11254 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760622052.115676   11254 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760622052.615022   11254 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760622052.615072   11254 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760622052.615081   11254 computation_placer.cc:177] computation placer alr

In [11]:
!python -m monai.apps.auto3dseg AutoRunner run \
    --input "/content/kits23/input.yaml" \
    --algos segresnet

2025-10-16 13:46:10.209695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760622370.472704   12621 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760622370.545006   12621 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760622371.083103   12621 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760622371.083148   12621 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760622371.083153   12621 computation_placer.cc:177] computation placer alr